In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ==========================================
# MEDI-SIMPLIFIER: CAPSTONE PROJECT
# Track: Agents for Good (Healthcare)
# ==========================================

# --- 1. INSTALLATION & SETUP ---
# Uncomment the lines below if running in a new environment
# !pip install -q langchain langchain-openai google-search-results

import os
import sys
from typing import List

# LangChain Imports
from langchain.agents import Tool, initialize_agent, AgentType
from langchain.memory import ConversationBufferWindowMemory
from langchain_openai import ChatOpenAI
from langchain_community.utilities import SerpAPIWrapper
from langchain.prompts import SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate

# --- 2. CONFIGURATION (API KEYS) ---
# ⚠️ REPLACE THESE WITH YOUR ACTUAL KEYS BEFORE RUNNING
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_KEY_HERE"
os.environ["SERPAPI_API_KEY"] = "YOUR_SERPAPI_KEY_HERE"

class MedicalInsightEngine:
    """
    A robust agentic system designed to interpret complex medical text.
    Architecture:
    - Logic Engine: GPT-4 (Orchestrator)
    - Tools: Real-time Web Search (Verification)
    - Memory: Rolling Window Buffer (Context Retention)
    """

    def __init__(self):
        self._validate_keys()
        self.llm = ChatOpenAI(temperature=0.1, model_name="gpt-4")
        self.memory = self._initialize_memory()
        self.tools = self._initialize_tools()
        self.agent = self._build_agent()
        
    def _validate_keys(self):
        """Ensures API keys are present to prevent runtime crashes."""
        if "YOUR_" in os.environ.get("OPENAI_API_KEY", ""):
            print("❌ ERROR: Please update your API Keys in the Configuration section.")
            sys.exit(1)

    def _initialize_memory(self):
        """
        Creates a 'Short-term Memory' buffer.
        We use k=5 to keep the last 5 exchanges in context, simulating
        a doctor remembering the immediate conversation history.
        """
        return ConversationBufferWindowMemory(
            memory_key="chat_history",
            k=5,
            return_messages=True
        )

    def _initialize_tools(self) -> List[Tool]:
        """
        Defines the external tools the agent can use.
        Using Google Search to verify latest medical guidelines.
        """
        search = SerpAPIWrapper()
        return [
            Tool(
                name="Clinical Verification Search",
                func=search.run,
                description=(
                    "CRITICAL: Use this tool to define medical terms, "
                    "check standard reference ranges (e.g., blood pressure), "
                    "or verify side effects. Do not guess."
                )
            )
        ]

    def _build_agent(self):
        """
        Constructs the ReAct (Reason + Act) Agent.
        """
        return initialize_agent(
            tools=self.tools,
            llm=self.llm,
            agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
            verbose=True, # Prints the "thought process" to the console
            memory=self.memory,
            handle_parsing_errors=True
        )

    def run_session(self, user_input: str):
        """
        The main entry point for processing a user query.
        """
        print(f"\n🔍 PROCESSING: '{user_input}'...")
        try:
            # We wrap the user input in a specific instruction to force formatting
            enhanced_prompt = (
                f"User Query: {user_input}\n\n"
                "INSTRUCTIONS:\n"
                "1. If this is medical text, search for the terms to verify them.\n"
                "2. Translate it into simple, 5th-grade English.\n"
                "3. Provide a 'Doctor Q&A Checklist' at the end.\n"
                "4. Be empathetic but factual."
            )
            
            response = self.agent.run(enhanced_prompt)
            return response
        except Exception as e:
            return f"System Error: {str(e)}"

# --- 3. EXECUTION SCENARIO ---

def run_demonstration():
    print("🏥 BOOTING MEDI-SIMPLIFIER AGENT...\n")
    
    # Instantiate the Engine
    engine = MedicalInsightEngine()
    
    # --- INTERACTION 1: ONBOARDING (Setting Context) ---
    print("--- [Step 1] Patient Onboarding ---")
    patient_profile = "I am a 65-year-old male with Type 2 Diabetes. I live alone."
    print(f"User: {patient_profile}")
    
    response_1 = engine.run_session(patient_profile)
    print(f"\n🤖 AGENT:\n{response_1}\n")
    print("="*60)

    # --- INTERACTION 2: COMPLEX INPUT (The Core Problem) ---
    print("\n--- [Step 2] Processing Medical Report ---")
    complex_report = (
        "I received my lab results. It says: "
        "'HbA1c is 8.5%, indicative of poor glycemic control. "
        "Microalbuminuria present. Initiate ACE inhibitor therapy.'"
    )
    print(f"User: {complex_report}")
    
    response_2 = engine.run_session(complex_report)
    
    print(f"\n🤖 AGENT RESPONSE:\n{response_2}")
    print("\n" + "="*60)
    
    # --- INTERACTION 3: MEDICATION INQUIRY (Follow up) ---
    print("\n--- [Step 3] Medication Safety Check ---")
    med_question = "My doctor mentioned 'Lisinopril'. Is that safe for my kidneys?"
    print(f"User: {med_question}")
    
    response_3 = engine.run_session(med_question)
    print(f"\n🤖 AGENT RESPONSE:\n{response_3}")

if __name__ == "__main__":
    run_demonstration()